In [1]:
import pypdf
import fitz # PyMuPDF

In [25]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Usando pypdf para PDFs nativos
        reader = pypdf.PdfReader(pdf_path)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text() + "\n"
    except Exception as e:
        print(f"Error con pypdf, intentando con PyMuPDF: {e}")
        try:
            # Usando PyMuPDF para más robustez o coordenadas
            doc = fitz.open(pdf_path)
            for page in doc:
                text += page.get_text("text") + "\n" # "text" para texto simple, "words" para coordenadas
            doc.close()
        except Exception as e:
            print(f"Error con PyMuPDF: {e}")
            # Si falla, podría ser un PDF escaneado
            # Aquí iría la lógica para OCR (ver siguiente sección)
    return text

def extract_text_from_pdf_v2(pdf_path):
    text = ""
    try:
        # Usando PyMuPDF para más robustez o coordenadas
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text("words") + "\n" # "text" para texto simple, "words" para coordenadas
        doc.close()
    except Exception as e:
        print(f"Error con PyMuPDF: {e}")
        # Si falla, podría ser un PDF escaneado
        # Aquí iría la lógica para OCR (ver siguiente sección)
    return text

In [26]:
texto = extract_text_from_pdf("Resolucion_UNO_2025-01-03 14_09_09.271.pdf")
print(texto)

incorrect startxref pointer(1)
parsing for Object Streams


JUZGADO PENAL UNIPERSONAL - SEDE NCPP EL AGUSTINO 
EXPEDIENTE : 00341-2024-1-3203-JR-PE-02 
JUEZ   : LOAYZA SANCHEZ RUTH KARINA 
ESPECIALISTA : CONDORI ACUÑA HERBERTH 
MINISTERIO PUBLICO : PRIMERA FISCALIA SEGUNDO DESPA CHO EL 
AGUSTINO  
REPRESENTANTE : SIGUAS ZEGARRA, MONICA DEL ROSARIO 
IMPUTADO : BRACAMONTE PEREZ, FREDDY SAMMY 
DELITO  : OMISIÓN DE ASISTENCIA FAMILIAR 
AGRAVIADO : BRACAMONTE SIGUAS, DYLAND CALEB 
 
RESOLUCIÓN NÚMERO UNO    
El Agustino, veintiséis de diciembre  
Del dos mil veinticuatro. - 
    
   AUTOS Y VISTOS : AVOCÁNDOSE  la suscrita conforme a 
lo dispuesto en la Resolución Administrativa N°152- 2022-P-CSJLE-PJ; y al 
oficio con ingreso número 202583-2024 Téngase por r ecibido el expediente 
principal proveniente del Segundo Juzgado de Invest igación Preparatoria 
de El Agustino, conteniendo la acusación presentada  por la Primera 
fiscalía provincial Penal Corporativa de El Agustin o – Segundo despacho 
de Investigación, y, ATENDIENDO :    
    
PRIMERO .- Q

In [27]:
import re
import spacy

In [28]:
# Cargar modelo de Spacy (ej. para español)
try:
    nlp = spacy.load("es_core_news_lg")
except OSError:
    print("Descargando modelo 'es_core_news_lg' de Spacy...")
    spacy.cli.download("es_core_news_lg")
    nlp = spacy.load("es_core_news_lg")


In [58]:
# Organizaciones o lugares que no son PII específicas.
EXCLUDED_GENERIC_ENTITIES = {
    "PODER JUDICIAL",
    "MINISTERIO PUBLICO",
    "CORTE SUPERIOR DE JUSTICIA",
    "JUZGADO DE PAZ LETRADO",
    "SALAS SUPERIORES",
    "CORTES SUPERIORES",
    "CORTE SUPREMA DE JUSTICIA",
    "FISCALÍA DE LA NACIÓN",
    "CONSEJO NACIONAL DE LA MAGISTRATURA", # Si aún se usa en documentos históricos
    "SUNARP",
    "RENIEC",
    "MINJUS", # Ministerio de Justicia y Derechos Humanos
    "PERÚ", # Si aparece como entidad LOC
    "LIMA", # Si aparece como entidad LOC y no es parte de una dirección específica
    # Añadir más entidades genéricas relevantes para el corpus
}

EXCLUDED_PROCESS_TERMS  = {
    "AUTOS",
    "VISTOS",
    "CONSIDERANDO",
    "CONSIDERANDOS",
    "RESUELVE",
    "POR TANTO",
    "DÉJESE CONSTANCIA",
    "SE RESUELVE",
    "ORDENA",
    "OMISIÓN",
    "SENTENCIA",
    "RESOLUCIÓN",
    "DELITO",
    "DECRETO",
    "LEGISLATIVO",
    "NOTIFICAR",
    # Añadir más términos según sea necesario
}

def find_pii(text):
    found_pii = []

    # 1. Regex para patrones comunes
    # Teléfono (ejemplo simple, ajustar a formatos específicos de Perú)
    phone_pattern = r'\b(?:\+?51)?\s?9\d{8}\b' # Adaptado para números de 9 dígitos que empiezan con 9 en Perú, opcional +51
    for match in re.finditer(phone_pattern, text):
        found_pii.append({'type': 'PHONE', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # Email
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    for match in re.finditer(email_pattern, text):
        found_pii.append({'type': 'EMAIL', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # DNI/CE (Ejemplo para Perú, ajustar patrón si es necesario)
    dni_pattern = r'\b[0-9]{8}\b' # 8 dígitos para DNI
    for match in re.finditer(dni_pattern, text):
        found_pii.append({'type': 'DNI', 'text': match.group(0), 'start': match.start(), 'end': match.end()})

    # RUC (ejemplo para Perú)
    ruc_pattern = r'\b(?:10|20)\d{9}\b' # Empieza con 10 o 20, 11 dígitos
    for match in re.finditer(ruc_pattern, text):
        found_pii.append({'type': 'RUC', 'text': match.group(0), 'start': match.start(), 'end': match.end()})
    

    # 2. Spacy para NER
    doc = nlp(text)
    for ent in doc.ents:
        # Normaliza el texto de la entidad para comparación (mayúsculas, sin espacios extra)
        normalized_ent_text = ent.text.strip().upper()

        # Filtrar tipos de entidades que nos interesan como PII (PER, LOC, ORG son los más comunes para PII)
        # MISC también puede ser PII, pero a menudo tiene falsos positivos.
        if ent.label_ in ["PER"]: 
            
            # 1. Excluir términos procesales específicos (ej. AUTOS, VISTOS)
            if normalized_ent_text in EXCLUDED_PROCESS_TERMS:
                print(f"DEBUG: Excluyendo término procesal '{ent.text}' ({ent.label_})")
                continue # Saltar esta entidad

            # 2. Excluir entidades genéricas (ej. nombres de instituciones)
            if normalized_ent_text in EXCLUDED_GENERIC_ENTITIES:
                print(f"DEBUG: Excluyendo entidad genérica '{ent.text}' ({ent.label_})")
                continue # Saltar esta entidad
            
            # 3. Reglas específicas para evitar falsos positivos con LOC y ORG
            # Si el modelo clasifica "Lima" como LOC pero no es parte de una dirección completa
            # Puedes necesitar un análisis de contexto más profundo aquí si "Lima" es PII o no
            if ent.label_ == "LOC" and normalized_ent_text == "LIMA":
                # Puedes añadir lógica para verificar si "Lima" está en una dirección o solo es el nombre de la ciudad
                # Por ahora, la excluimos si es solo "Lima" para reducir falsos positivos.
                print(f"DEBUG: Excluyendo LOC genérico '{ent.text}'")
                continue

            # Si pasa los filtros, es una PII válida
            found_pii.append({'type': ent.label_, 'text': ent.text, 'start': ent.start_char, 'end': ent.end_char})

    return found_pii

In [59]:
pii_lista = find_pii(texto)
print(pii_lista)

DEBUG: Excluyendo entidad genérica 'MINISTERIO PUBLICO' (PER)
[{'type': 'PHONE', 'text': ' 965403382', 'start': 3364, 'end': 3374}, {'type': 'PER', 'text': 'LOAYZA', 'start': 101, 'end': 107}, {'type': 'PER', 'text': 'RUTH KARINA \nESPECIALISTA', 'start': 116, 'end': 141}, {'type': 'PER', 'text': 'OMISIÓN DE', 'start': 344, 'end': 354}, {'type': 'PER', 'text': 'Segundo Juzgado de Invest igación Preparatoria \nde El Agustino', 'start': 754, 'end': 816}, {'type': 'PER', 'text': 'Segundo \nJuzgado de Investigación Preparatoria de El Agustin', 'start': 1186, 'end': 1246}, {'type': 'PER', 'text': 'Dyland Caled Bracamonte Siguas', 'start': 1627, 'end': 1657}, {'type': 'PER', 'text': 'Mónica del Rosario Siguas Zegarra', 'start': 1687, 'end': 1720}, {'type': 'PER', 'text': 'inconcur rencia', 'start': 4447, 'end': 4462}, {'type': 'PER', 'text': 'E.  EXHORTAR', 'start': 4572, 'end': 4584}, {'type': 'PER', 'text': 'G.  ESTABLECER', 'start': 5031, 'end': 5045}, {'type': 'PER', 'text': 'H.  NOTIFIC

In [60]:
def anonymize_text_with_x(original_text, found_pii):
    """
    Reemplaza las entidades PII en el texto original con el carácter 'X'.

    Args:
        original_text (str): El texto completo extraído del PDF.
        found_pii (list): Una lista de diccionarios, donde cada diccionario
                          representa una PII encontrada y tiene las claves
                          'start' (índice de inicio) y 'end' (índice de fin).
                          Ejemplo: [{'type': 'PERSON', 'text': 'Juan Pérez', 'start': 10, 'end': 20}]

    Returns:
        str: El texto con las PII reemplazadas por 'X'.
    """
    # Convertir el texto a una lista de caracteres para facilitar la modificación in-place
    # Esto es más eficiente que manipular cadenas repetidamente.
    text_chars = list(original_text)

    # Es crucial procesar las PII en orden inverso de sus posiciones
    # para que los reemplazos no afecten los índices de las PII aún no procesadas.
    # Alternativamente, se puede construir un nuevo texto.
    # Para este método de lista de caracteres, el orden no es tan crítico
    # si solo estamos reemplazando con 'X' de la misma longitud.
    # Sin embargo, si hubiera solapamientos, el orden inverso o un enfoque de "marcado"
    # sería mejor. Para simple reemplazo, podemos iterar directamente.

    for pii in found_pii:
        start_index = pii['start']
        end_index = pii['end']

        # Asegurarse de que los índices estén dentro de los límites del texto
        if 0 <= start_index < len(text_chars) and 0 <= end_index <= len(text_chars) and start_index < end_index:
            # Calcular la longitud de la PII
            length_of_pii = end_index - start_index
            # Reemplazar el segmento de texto con 'X'
            for i in range(start_index, end_index):
                text_chars[i] = 'X'
        else:
            print(f"Advertencia: Índices de PII inválidos o fuera de rango: {pii}. No se anonimizará.")

    # Unir la lista de caracteres de nuevo en una cadena
    anonymized_text = "".join(text_chars)
    return anonymized_text



In [61]:
texto_anonimizado = anonymize_text_with_x(texto, pii_lista)

In [62]:
print(texto_anonimizado)

JUZGADO PENAL UNIPERSONAL - SEDE NCPP EL AGUSTINO 
EXPEDIENTE : 00341-2024-1-3203-JR-PE-02 
JUEZ   : XXXXXX SANCHEZ XXXXXXXXXXXXXXXXXXXXXXXXX : CONDORI ACUÑA HERBERTH 
MINISTERIO PUBLICO : PRIMERA FISCALIA SEGUNDO DESPA CHO EL 
AGUSTINO  
REPRESENTANTE : SIGUAS ZEGARRA, MONICA DEL ROSARIO 
IMPUTADO : BRACAMONTE PEREZ, FREDDY SAMMY 
DELITO  : XXXXXXXXXX ASISTENCIA FAMILIAR 
AGRAVIADO : BRACAMONTE SIGUAS, DYLAND CALEB 
 
RESOLUCIÓN NÚMERO UNO    
El Agustino, veintiséis de diciembre  
Del dos mil veinticuatro. - 
    
   AUTOS Y VISTOS : AVOCÁNDOSE  la suscrita conforme a 
lo dispuesto en la Resolución Administrativa N°152- 2022-P-CSJLE-PJ; y al 
oficio con ingreso número 202583-2024 Téngase por r ecibido el expediente 
principal proveniente del XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX, conteniendo la acusación presentada  por la Primera 
fiscalía provincial Penal Corporativa de El Agustin o – Segundo despacho 
de Investigación, y, ATENDIENDO :    
    
PRIMERO .- Q